In [20]:
import pytesseract
import cv2
from PIL import Image
import pandas as pd
import numpy as np

In [21]:
path = 'C:\\Users\\kakut\\OneDrive\\Desktop\\ocr images\\'
image_file = f'{path}photo_2024-02-20_16-52-48.jpg'
img = cv2.imread(image_file)

In [22]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imwrite(f"{path}\\temp\\index_gray.png", gray)


True

In [23]:
_, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)
cv2.imwrite(f"{path}\\temp\\index_thresh.png", thresh)

True

In [24]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
dilated = cv2.dilate(thresh, kernel, iterations=5)
cv2.imwrite(f"{path}\\temp\\index_dilated.png", dilated)

True

In [25]:
eroded = cv2.erode(dilated, kernel, iterations=1)
cv2.imwrite(f"{path}\\temp\\index_eroded.png", eroded)

True

In [26]:
cnts = cv2.findContours(eroded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])


In [31]:
results  = []
rois = []
for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    if h > 200 and w > 20:
        roi = img[y:y+h, x:x+w]
        rois.append(roi)
        cv2.rectangle(img, (x, y), (x+w, y+h), (36, 255, 12), 2)
        ocr_result = pytesseract.image_to_string(roi)
        ocr_result = ocr_result.split("\n")
        for item in ocr_result:
            results.append(item)
cv2.imwrite(f"{path}\\temp\\index_bbox_new.png", img)

True

In [39]:
# Extract text from the first and fifth ROIs
roi1_text = pytesseract.image_to_string(rois[1])
roi4_text = pytesseract.image_to_string(rois[4])

# Strings to exclude from the extracted text
exclude_strings = ("Marco's Pizza Store","Marco's Pizza store")

# Combine text from both ROIs and filter out specified strings
col1 = [i for i in roi1_text.split("\n") + roi4_text.split("\n") if i.strip() and i not in exclude_strings]

# Get the length of the combined list
len(col1)

24

In [51]:
col2 = ['C','C','C','M','C','C','M','C','C','C','M','M','C','C','C','C','C','M','M','C','M','C','C','M',]
df = pd.DataFrame({'Store':col1,'Ownership Type':col2})
df

,Store,Ownership Type
0,2607 Starr Av,C
1,309 W Alexis Rd,C
2,2652 W Central Av,C
3,630 Dixie Hwy,M
4,3510 Dorr St,C
5,5055 Glendale Av,C
6,3382 Lagrange St,M
7,2525 W Laskey Rd,C
8,303 Louisiana Av,C
9,149 Main St,C


In [52]:
random_samples = df.sample(n=5)
random_samples

,Store,Ownership Type
5,5055 Glendale Av,C
14,3678 Rugby Dr,C
6,3382 Lagrange St,M
19,106 E Airport Hwy,C
21,4252 Monroe,C


Stratified sampling

In [54]:
import pandas as pd

# Assuming df is your DataFrame containing the 'Salad bar' column

# Sample two rows where 'Salad bar' is "C"
sample_c = df[df['Ownership Type'] == "C"].sample(n=2)

# Sample one row where 'Salad bar' is "M"
sample_m = df[df['Ownership Type'] == "M"].sample(n=1)

# Concatenate the two sampled DataFrames
sample = pd.concat([sample_c, sample_m])

sample

,Store,Ownership Type
22,2036 Woodville Rd,C
5,5055 Glendale Av,C
18,5155 S Main,M
